Plot Tidal Spectra
==================

This ({nb-download}`notebook <Plot-Tidal-Spectra.ipynb>`) demonstrates plotting the tidal spectra of potential amplitudes for semi-diurnal and diurnal constituents following [Munk and Cartwright (1966)](https://doi.org/10.1098/rsta.1966.0024), [Cartwright and Tayler (1971)](https://doi.org/10.1111/j.1365-246X.1971.tb01803.x) and [Cartwright and Edden (1973)](https://doi.org/10.1111/j.1365-246X.1973.tb03420.x)
        
## Python Dependencies
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [matplotlib: Python 2D plotting library](http://matplotlib.org/)  

## Program Dependencies

- `astro.py`: computes the basic astronomical mean longitudes
- `arguments.py`: load the nodal corrections for tidal constituents

## Load modules

In [ ]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.offsetbox as offsetbox

# import tide programs
import pyTMD.astro
import pyTMD.arguments

# autoreload
%load_ext autoreload
%autoreload 2

## Plot Cartwright-Tayler-Edden Tables

In [ ]:
# create figure axis
fig, ax = plt.subplots(ncols=2, sharey=True, figsize=(10, 5))
# run for degree-2 constituents
l = 2
# parse the Cartwright and Edden (1973) table for degree-2 constituents
CTE = pyTMD.arguments._parse_tide_potential_table(
    pyTMD.arguments._ce1973_table_1)
# major constituents to label
scons = ['2n2', 'm2', 's2', 'n2']
dcons = ['q1', 'o1', 'k1', 'j1']
# for each spectral line
for i, line in enumerate(CTE):
    # spherical harmonic dependence (order)
    TAU = line['tau']
    # Doodson coefficients for constituent
    S = line['s']
    H = line['h']
    P = line['p']
    # convert N for ascending lunar node
    N = -1.0*line['n']
    PP = line['pp']
    # use cosines for (m + n) even
    # and sines for (m + n) odd
    K = -1.0*np.mod(l + TAU, 2)
    # determine constituent phase using equilibrium arguments
    arguments = np.array([TAU, S, H, P, N, PP, K], dtype=np.float64)
    # calculate the angular frequency
    omega = pyTMD.arguments._frequency(arguments, method='ASTRO5')
    # convert to cycles per solar day
    f = omega*86400.0/(2.0*np.pi)
    # amplitude in cm
    amp = 100.0*np.abs(line['Hs3'])
    # get the constituent ID based on the first 6 arguments
    cons = pyTMD.arguments._to_constituent_id(arguments,
        arguments=6, raise_error=False)
    # plot amplitudes and color if in the major constituents list
    if cons in scons:
        ax[0].semilogy([f, f], [0.0, amp], 'r', zorder=1)
    elif cons in dcons:
        ax[1].semilogy([f, f], [0.0, amp], 'r', zorder=1)
    elif (f >= 1.75) and (f <= 2.10):
        ax[0].semilogy([f, f], [0.0, amp], '0.4', zorder=0)
    elif (f >= 0.80) and (f <= 1.15):
        ax[1].semilogy([f, f], [0.0, amp], '0.4', zorder=0)

# run for degree-3 constituents
l = 3
# parse the Cartwright and Tayler (1971) table for degree-3 constituents
CTE = pyTMD.arguments._parse_tide_potential_table(
    pyTMD.arguments._ct1971_table_5)
# for each line in the table
for i, line in enumerate(CTE):
    # spherical harmonic dependence (order)
    TAU = line['tau']
    # Doodson coefficients for constituent
    S = line['s']
    H = line['h']
    P = line['p']
    # convert N for ascending lunar node
    N = -1.0*line['n']
    PP = line['pp']
    # use cosines for (l + tau) even
    # and sines for (l + tau) odd
    K = -1.0*np.mod(l + TAU, 2)
    # determine constituent phase using equilibrium arguments
    arguments = np.array([TAU, S, H, P, N, PP, K], dtype=np.float64)
    # calculate the angular frequency
    omega = pyTMD.arguments._frequency(arguments, method='ASTRO5')
    # convert to frequency (solar days per cycle)
    f = np.abs(omega*86400.0)/(2.0*np.pi)
    # amplitude in cm
    amp = 100.0*np.abs(line['Hs3'])
    # plot amplitudes
    if (f >= 1.75) and (f <= 2.10):
        ax[0].semilogy([f, f], [0.0, amp], '0.4', zorder=0)
    elif (f >= 0.80) and (f <= 1.15):
        ax[1].semilogy([f, f], [0.0, amp], '0.4', zorder=0)
    

# add labels for some major semi-diurnal constituents
for c in scons:
    omega = pyTMD.arguments.frequency(c)
    f = omega*86400.0/(2.0*np.pi)
    ax[0].text(f, 100.0, c, color='r', fontsize=10, ha='center')
# add labels for some major diurnal constituents
for c in dcons:
    omega = pyTMD.arguments.frequency(c)
    f = omega*86400.0/(2.0*np.pi)
    ax[1].text(f, 100.0, c, color='r', fontsize=10, ha='center')

# axes parameters
xlabels = ['Frequency [cpd]', 'Frequency [cpd]']
ylabels = ['Amplitude [cm]', '']
xlimits = [[1.75, 2.10], [0.80, 1.15]]
titles = ['Semi-diurnal', 'Diurnal']
labels = ['a)', 'b)']
# adjust axes
for i, axis in enumerate(ax):
    # add x and y labels
    axis.set_xlabel(xlabels[i])
    axis.set_ylabel(ylabels[i])
    # set limits
    axis.set_xlim(xlimits[i])
    axis.set_ylim(1e-3, 2e2)
    # set titles
    axis.set_title(f'{titles[i]} Tidal Spectra')
    # add labels
    at = offsetbox.AnchoredText(labels[i], loc=2, frameon=False,
        prop=dict(size=14,weight='bold'))
    axis.axes.add_artist(at)
    # adjust axes
    axis.get_xaxis().set_tick_params(which='both', direction='in')
    axis.get_yaxis().set_tick_params(which='both', direction='in')
# adjust subplots
fig.subplots_adjust(left=0.07, right=0.97, bottom=0.10, top=0.95, wspace=0.10)
plt.show()